In [ ]:
#query for efii2 deficit for skin ulcer
 CREATE OR REPLACE TABLE `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_efi2_skin_ulcer` as (
#bring all snomed and ctv3codes for care home cohort in primary into the cte
with a as (
  select
  person_id,
  snomedcode,
  ctv3code,
  dateevent,
  cast(numericvalue as bignumeric) as numeric_value
  from `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)),
  #join care home entry date from msater table
  a_master as (
    select
    a.*,
    cast(j.first_episodestartdate as datetime) first_episodestartdate
    from a
    left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` j on
    a.person_id = j.person_id),
#filter for ski ulcer snomed codes without numericvalues
  b as (
    select
    person_id,
    snomedcode,
    dateevent
    from a_master
    where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Skin ulcer' AND Otherinstructions IS NULL) AND dateevent < first_episodestartdate),
#filter forskin ulcer with snomed codes 
  c as (
    select
    person_id,
    snomedcode, 
    dateevent
    from a_master
    where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Skin ulcer' AND Otherinstructions = 'If greater than 0') AND dateevent < first_episodestartdate AND numeric_value > 0),

  d as (select
    person_id,
    snomedcode, 
    dateevent
    from a_master
    where snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Skin ulcer' AND Otherinstructions = 'Any value') AND dateevent < first_episodestartdate AND numeric_value IS NOT NULL),
e as (select
    person_id,
    ctv3code, 
    dateevent
    from a_master
    where ctv3code in(select CTV3 from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Skin ulcer' and Otherinstructions IS NULL) and dateevent < first_episodestartdate
),
#union distinct individuals identified from ctv3 and snomed ct codes together
  snomed_ctv3 as (
    select
    * from b

    union distinct

    select *
    from c
    
    union distinct 
    
    select *
    from d
    
    union distinct
    
    select *
    from e)
#apply dummy variable
    select
    distinct person_id,
    case when dateevent IS NOT NULL then 'yes'
    end as skin_ulcer
    from snomed_ctv3)
  



